In [ ]:
!pip install kaggle

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c santander-customer-transaction-prediction
!unzip data-science-for-good.zip
!unzip test.csv.zip
!unzip train.csv.zip

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 66 bytes
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
unzip:  cannot find or open data-science-for-good.zip, data-science-for-good.zip.zip or data-science-for-good.zip.ZIP.
Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from matplotlib import pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from scipy.special import erfinv
import time
import lightgbm as lgb
from sklearn.naive_bayes import GaussianNB
!pip install optuna
import optuna
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.optim import RMSprop
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from torch.optim.lr_scheduler import CyclicLR
from sklearn.preprocessing import StandardScaler

In [ ]:
pd.set_option('display.max_columns', None)

train_data = pd.read_csv(r"train.csv")
test_data = pd.read_csv(r"test.csv")

train_X = train_data.drop(columns=['ID_code','target'])
train_y = train_data['target']
test_X = test_data.drop(columns=['ID_code'])

seed = 12

In [ ]:
train_data

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,4.6667,3.8743,-5.2387,7.3746,11.5767,12.0446,11.6418,-7.0170,5.9226,-14.2136,16.0283,5.3253,12.9194,29.0460,-0.6940,5.1736,-0.7474,14.8322,11.2668,5.3822,2.0183,10.1166,16.1828,4.9590,2.0771,-0.2154,8.6748,9.5319,5.8056,22.4321,5.0109,-4.7010,21.6374,0.5663,5.1999,8.8600,43.1127,18.3816,-2.3440,23.4104,6.5199,12.1983,13.6468,13.8372,1.3675,2.9423,-4.5213,21.4669,9.3225,16.4597,7.9984,-1.7069,-21.4494,6.7806,11.0924,9.9913,14.8421,0.1812,8.9642,16.2572,2.1743,-3.4132,9.4763,13.3102,26.5376,1.4403,14.7100,6.0454,9.5426,17.1554,14.1104,24.3627,2.0323,6.7602,3.9141,-0.4851,2.5240,1.5093,2.5516,15.5752,-13.4221,7.2739,16.0094,9.7268,0.8897,0.7754,4.2218,12.0039,13.8571,-0.7338,-1.9245,15.4462,12.8287,0.3587,9.6508,6.5674,5.1726,3.1345,29.4547,31.4045,2.8279,15.6599,8.3307,-5.6011,19.0614,11.2663,8.6989,8.3694,11.5659,-16.4727,4.0288,17.9244,18.5177,10.7800,9.0056,16.6964,10.4838,1.6573,12.1749,-13.1324,17.6054,11.5423,15.4576,5.3133,3.6159,5.0384,6.6760,12.6644,2.7004,-0.6975,9.5981,5.4879,-4.7645,-8.4254,20.8773,3.1531,18.5618,7.7423,-10.1245,13.7241,-3.5189,1.7202,-8.4051,9.0164,3.0657,14.3691,25.8398,5.8764,11.8411,-19.7159,17.5743,0.5857,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,-0.4032,8.0585,14.0239,8.4135,5.4345,13.7003,13.8275,-15.5849,7.8000,28.5708,3.4287,2.7407,8.5524,3.3716,6.9779,13.8910,-11.7684,-2.5586,5.0464,0.5481,-9.2987,7.8755,1.2859,19.3710,11.3702,0.7399,2.7995,5.8434,10.8160,3.6783,-11.1147,1.8730,9.8775,11.7842,1.2444,-47.3797,7.3718,0.1948,34.4014,25.7037,11.8343,13.2256,-4.1083,6.6885,-8.0946,18.5995,19.3219,7.0118,1.9210,8.8682,8.0109,-7.2417,1.7944,-1.3147,8.1042,1.5365,5.4007,7.9344,5.0220,2.2302,40.5632,0.5134,3.1701,20.1068,7.7841,7.0529,3.2709,23.4822,5.5075,13.7814,2.5462,18.1782,0.3683,-4.8210,-5.4850,13.7867,-13.5901,11.0993,7.9022,12.2301,0.4768,6.8852,8.0905,10.9631,11.7569,-1.2722,24.7876,26.6881,1.8944,0.6939,-13.6950,8.4068,35.4734,1.7093,15.1866,2.6227,7.3412,32.0888,13.9550,13.0858,6.6203,7.1051,5.3523,8.5426,3.6159,4.1569,3.0454,7.8522,-11.5100,7.5109,31.5899,9.5018,8.2736,10.1633,0.1225,12.5942,14.5697

In [ ]:
### feature engineering: 1. count encoding 2. rank gaussian for the counts only
feats = list(train_X.columns)

data = pd.concat([train_X, test_X], axis=1)

def count_encoding(df):
  df_ = pd.DataFrame()
  feats = list(df.columns)
  for feat in tqdm(feats):
      val_cts = df[feat].value_counts().to_dict()
      ## there's no NA data - don't do anything
      df_['{}_ct'.format(feat)] = df[feat].map(val_cts)
  return df_

def rank_gauss(df, cols=None):
    ## rank -> (0,1)
    df_ = (df.rank() - 1) / len(df)
    ## (-1,1)
    df_ = df_ * 2 - 1
    df_ = df_.clip(-0.999, 0.999)
    return erfinv(df_)
    
def rank_gauss_scaler(test_df, train_df):
    train_ct = count_encoding(train_df)
    test_ct = count_encoding(test_df)
    
    train_df_encoded = rank_gauss(train_ct)
    
    # training set and test set should have the same set of columns
    ## TODO: assert
    feats = list(train_ct.columns)
    rank_dict = {}
    for feat in feats:
        rank_dict[feat] = dict(zip(train_ct[feat], train_df_encoded[feat]))
    return train_df_encoded, rank_dict

cols = list(train_data.columns)
cols.remove('ID_code')
cols.remove('target')
ct_cols = [col + '_ct' for col in cols]

scaler = StandardScaler()
train_X_count = rank_gauss(count_encoding(train_X))
test_X_count = rank_gauss(count_encoding(test_X))
train_X_encoded = pd.DataFrame(scaler.fit_transform(train_X), index=train_X.index, columns=train_X.columns)
test_X_encoded = pd.DataFrame(scaler.fit_transform(test_X), index=test_X.index, columns=test_X.columns)

train_X_encoded = pd.concat([train_X_encoded, train_X_count], axis=1)
test_X_encoded = pd.concat([test_X_encoded, test_X_count], axis=1)

100%|██████████| 200/200 [00:30<00:00,  6.57it/s]


In [ ]:
train_X_encoded.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_0_ct,var_1_ct,var_2_ct,var_3_ct,var_4_ct,var_5_ct,var_6_ct,var_7_ct,var_8_ct,var_9_ct,var_10_ct,var_11_ct,var_12_ct,var_13_ct,var_14_ct,var_15_ct,var_16_ct,var_17_ct,var_18_ct,var_19_ct,var_20_ct,var_21_ct,var_22_ct,var_23_ct,var_24_ct,var_25_ct,var_26_ct,var_27_ct,var_28_ct,var_29_ct,var_30_ct,var_31_ct,var_32_ct,var_33_ct,var_34_ct,var_35_ct,var_36_ct,var_37_ct,var_38_ct,var_39_ct,var_40_ct,var_41_ct,var_42_ct,var_43_ct,var_44_ct,var_45_ct,var_46_ct,var_47_ct,var_48_ct,var_49_ct,var_50_ct,var_51_ct,var_52_ct,var_53_ct,var_54_ct,var_55_ct,var_56_ct,var_57_ct,var_58_ct,var_59_ct,var_60_ct,var_61_ct,var_62_ct,var_63_ct,var_64_ct,var_65_ct,var_66_ct,var_67_ct,var_68_ct,var_69_ct,var_70_ct,var_71_ct,var_72_ct,var_73_ct,var_74_ct,var_75_ct,var_76_ct,var_77_ct,var_78_ct,var_79_ct,var_80_ct,var_81_ct,var_82_ct,var_83_ct,var_84_ct,var_85_ct,var_86_ct,var_87_ct,var_88_ct,var_89_ct,var_90_ct,var_91_ct,var_92_ct,var_93_ct,var_94_ct,var_95_ct,var_96_ct,var_97_ct,var_98_ct,var_99_ct,var_100_ct,var_101_ct,var_102_ct,var_103_ct,var_104_ct,var_105_ct,var_106_ct,var_107_ct,var_108_ct,var_109_ct,var_110_ct,var_111_ct,var_112_ct,var_113_ct,var_114_ct,var_115_ct,var_116_ct,var_117_ct,var_118_ct,var_119_ct,var_120_ct,var_121_ct,var_122_ct,var_123_ct,var_124_ct,var_125_ct,var_126_ct,var_127_ct,var_128_ct,var_129_ct,var_130_ct,var_131_ct,var_132_ct,var_133_ct,var_134_ct,var_135_ct,var_136_ct,var_137_ct,var_138_ct,var_139_ct,var_140_ct,var_141_ct,var_142_ct,var_143_ct,var_144_ct,var_145_ct,var_146_ct,var_147_ct,var_148_ct,var_149_ct,var_150_ct,var_151_ct,var_152_ct,var_153_ct,var_154_ct,var_155_ct,var_156_ct,var_157_ct,var_158_ct,var_159_ct,var_160_ct,var_161_ct,var_162_ct,var_163_ct,var_164_ct,var_165_ct,var_166_ct,var_167_ct,var_168_ct,var_169_ct,var_170_ct,var_171_ct,var_172_ct,var_173_ct,var_174_ct,var_175_ct,var_176_ct,var_177_ct,var_178_ct,var_179_ct,var_180_ct,var_181_ct,var_182_ct,var_183_ct,var_184_ct,var_185_ct,var_186_ct,var_187_ct,var_188_ct,var_189_ct,var_190_ct,var_191_ct,var_192_ct,var_193_ct,var_194_ct,var_195_ct,var_196_ct,var_197_ct,var_198_ct,var_199_ct
0,-0.577102,-1.273737,0.451707,-0.833709,0.235571,-0.536430,-0.334926,0.608751,-1.561580,-1.473796,0.460091,1.076623,-0.054077,-1.714773,0.561098,-0.009778,-1.401636,-0.229802,-1.395940,2.285292,-0.468989,-0.126734,-0.606166,-1.039947,1.005683,-0.823003,-0.184673,0.479492,-0.768964,0.360340,0.926437,1.162587,-1.299824,0.489773,-0.447275,1.300500,-0.484245,0.897088,-1.396490,0.789254,0.179582,-0.324223,0.509123,1.533

In [ ]:
### Stratified CV
k = 10
k_fold = StratifiedKFold(n_splits=k, shuffle=True, random_state=12)

In [ ]:
# Neural Network
#https://www.kaggle.com/super13579/pytorch-nn-cyclelr-k-fold-0-897-lightgbm-0-899
# since the variables are independent of each other, we set input dim of the first nn layer to 1

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Net(nn.Module):
    def __init__(self, input_size, hidden_dim, dropout):
        super(Net, self).__init__()
        layers = []
        layers.append(nn.Linear(1, hidden_dim))
        layers.append(nn.PReLU())
        layers.append(nn.Dropout(p=dropout))
        
        self.out_layer = nn.Linear(input_size * hidden_dim, 1)
        self.layers = nn.Sequential(*layers)
    
    def forward(self, X):
        N = X.shape[0]
        X_ = X.reshape(-1,1)
        out = self.layers(X_)
        out = out.reshape((N, -1))
        return torch.flatten(self.out_layer(out))
    
def evaluate(model, dataloader):
    model.eval()
    dataset = dataloader.dataset[:]
    inputs, labels = dataset[0], dataset[1]
    inputs = inputs.to(DEVICE)
    
    with torch.no_grad():
        outputs = model(inputs)
        pred_proba = sigmoid(outputs.detach().cpu().numpy())
        roc_auc = roc_auc_score(labels.detach().cpu().numpy(), pred_proba)
  
    model.train()
    return roc_auc

def sigmoid(x):
  return 1/(1+np.exp(-x))

In [ ]:
import gc
import time

n_epoch = 30
batch_size = 256
hidden_dim = 24
dropout = 0.6
lr = 5e-4
weight_decay = 1e-5
    
criterion = torch.nn.BCEWithLogitsLoss()
cv_scores = np.empty(k)
# need to turn cycle_momentum to False for Adam

patience = 10

nn_pred = np.zeros(len(test_X_encoded))
cv_scores = np.empty(k)
### 10 fold CV
for fold_n, (train_index, valid_index) in enumerate(k_fold.split(train_X_encoded, train_y)):
  print('Fold', fold_n, 'started at', time.ctime())
  net = Net(train_X_encoded.shape[1], hidden_dim, dropout).to(DEVICE)
  optimizer = Adam(net.parameters(), lr=lr, weight_decay=weight_decay)
  scheduler = CyclicLR(optimizer, base_lr=lr, max_lr=5e-3, cycle_momentum=False)
        
  X_train, X_valid = train_X_encoded.iloc[train_index], train_X_encoded.iloc[valid_index]
  y_train, y_valid = train_y.iloc[train_index], train_y.iloc[valid_index]
        
  X_train = torch.tensor(X_train.values, dtype=torch.float)
  X_valid = torch.tensor(X_valid.values, dtype=torch.float)
  y_train = torch.tensor(y_train.values, dtype=torch.float)
  y_valid = torch.tensor(y_valid.values, dtype=torch.float)
  X_test = torch.tensor(test_X_encoded.values, dtype=torch.float)
        
  train = torch.utils.data.TensorDataset(X_train, y_train)
  valid = torch.utils.data.TensorDataset(X_valid, y_valid)
        
  train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
  valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
  stopped = False

  step = 0
  best_auc_roc = 0
  best_pred_proba = np.zeros(len(test_X_encoded))
  for epoch in np.arange(n_epoch):
    start_time = time.time()
    if step > patience:
      # store current best model
      cv_scores[fold_n] = best_auc_roc
      nn_pred = nn_pred +  best_pred_proba/k
      gc.collect()
      stopped = True
      print("training for epoch {} completed. Best auc_roc score: {:.4f}".format(epoch, best_auc_roc))
      break
      
    for i, data in enumerate(train_loader, 0):
      X_train, y_train = data
      X_train = X_train.to(DEVICE)
      y_train = y_train.to(DEVICE)
      optimizer.zero_grad()
                
      # forward
      y_pred = net(X_train)
      # loss
      loss =  criterion(y_pred, y_train)
      loss.backward()
      optimizer.step()
      
      

      if scheduler is not None:
          scheduler.step()
    
    # evaluate model
    train_auc_roc= evaluate(net, train_loader)
    valid_auc_roc= evaluate(net, valid_loader)
                
    if best_auc_roc < valid_auc_roc:
      best_auc_roc = valid_auc_roc
      X_test_cuda = X_test.to(DEVICE)
      best_pred_proba = sigmoid(net(X_test_cuda).detach().cpu().numpy())
      step = 0
    else:
      step = step + 1


    print("Epoch {} elapsed {} secs".format(epoch+1, time.time() - start_time))
    print("Train auc_roc: {:.4f}, Validation auc_roc: {:.4f},  Best Validation auc_roc: {:.4f}".format(train_auc_roc, valid_auc_roc, best_auc_roc))
    print("-----------------------------------------------------------------------------------")
  
  if stopped is False:
    cv_scores[fold_n] = best_auc_roc
    nn_pred = nn_pred + best_pred_proba/k
    
print("Training Complete, CV Score: {:.4f}".format(np.mean(cv_scores)))
nn_submission = pd.DataFrame({'ID_code': test_data.ID_code, 'target': nn_pred})
nn_submission.to_csv('submission.csv', index=False)

Fold 0 started at Tue Nov  9 16:12:30 2021
Epoch 1 elapsed 22.253979921340942 secs
Train auc_roc: 0.8787, Validation auc_roc: 0.8735,  Best Validation auc_roc: 0.8735
-----------------------------------------------------------------------------------
Epoch 2 elapsed 21.459531545639038 secs
Train auc_roc: 0.8910, Validation auc_roc: 0.8846,  Best Validation auc_roc: 0.8846
-----------------------------------------------------------------------------------
Epoch 3 elapsed 22.39742660522461 secs
Train auc_roc: 0.8934, Validation auc_roc: 0.8864,  Best Validation auc_roc: 0.8864
-----------------------------------------------------------------------------------
Epoch 4 elapsed 21.464014291763306 secs
Train auc_roc: 0.8961, Validation auc_roc: 0.8889,  Best Validation auc_roc: 0.8889
-----------------------------------------------------------------------------------
Epoch 5 elapsed 22.8403217792511 secs
Train auc_roc: 0.8981, Validation auc_roc: 0.8902,  Best Validation auc_roc: 0.8902
----

In [ ]:
nn_submission.to_csv('NN_submission_1.csv', index=False)

In [ ]:
# submit the file to kaggle
!kaggle competitions submit santander-customer-transaction-prediction -f NN_submission_1.csv -m "Neural Network with no iteraction between vars in the first layer"

100% 6.05M/6.05M [00:02<00:00, 2.47MB/s]
Successfully submitted to Santander Customer Transaction Prediction

In [ ]:
nn_submission

,ID_code,target
0,test_0,0.171988
1,test_1,0.155577
2,test_2,0.188781
3,test_3,0.170015
4,test_4,0.031101
...,...,...
199995,test_199995,0.022833
199996,test_199996,0.024966
199997,test_199997,0.003583
199998,test_199998,0.046593
